In [1]:
1-12 best - lightgbm

from src.preprocessors.add_columns import add_sin_cos_day, add_sin_cos_hour, add_ghi

def feature_engineering(df):
#     df = add_sin_cos_day(df)
#     df = add_sin_cos_hour(df)
    df = add_ghi(df)

    df["TARGET_ROLLING_MEAN_3"] = df["TARGET"].rolling(3).mean()
    df["TARGET_ROLLING_MEAN_5"] = df["TARGET"].rolling(5).mean()
    df["TARGET_ROLLING_MEAN_11"] = df["TARGET"].rolling(11).mean()
    df["TARGET_ROLLING_MEAN_23"] = df["TARGET"].rolling(23).mean()
    df["TARGET_ROLLING_MEAN_47"] = df["TARGET"].rolling(47).mean()
    
    scaled_rh = (df["RH"]) / 100
    _mean = df["T"].rolling(96).mean()
    _std = df["T"].rolling(96).std()
    df["SCALED_RH"] = scaled_rh * (df["T"] - _mean) / _std
    df["RH_CROSS_T"] = df["RH"] * df["T"]
    
    df.drop(["Day", "Hour", "Minute", "WS", "RH", "T"], axis=1, inplace=True)
    
    return df


import importlib
import src.loaders.data_loader
importlib.reload(src.loaders.data_loader)

from src.loaders.data_loader import generate_test_data, load_submission_data

submission_df = load_submission_data()

r = 0

for i in range(1, 10):
    q = i/10
    result = []
    param = {
    'application': 'quantile',
    'learning_rate': 0.1,
    'max_depth': 7,
    'num_leaves': 64,
    'n_jobs': 17,
    'seed': 42,
    'bagging_freq': 100,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'drop_rate': 0.2,
    'early_stopping_round': 10,
    'alpha': q,
    'boosting': 'dart',
    }
    
    num_round = 5000
    bst_1 = lgb.train(param, train_1_dataset, num_round, valid_sets=[valid_1_dataset])
    bst_2 = lgb.train(param, train_2_dataset, num_round, valid_sets=[valid_2_dataset])
    
    y_pred_1 = bst_1.predict(test_x)
    y_pred_2 = bst_2.predict(test_x)
    
    r += np.mean(np.abs(test_y_1 - y_pred_1) * q + np.abs(test_y_2 - y_pred_2) * q)
    
    for td in generate_test_data():
        td = feature_engineering(td)
        td = td.values.reshape(7, 48, -1)[(7 - x_day):, :, :].transpose(1, 0, 2).reshape(48, -1)
        result.append(bst_1.predict(td))
        result.append(bst_2.predict(td))
    
    submission_df[f"q_{q}"] = np.array(result).reshape(-1)

SyntaxError: invalid syntax (<ipython-input-1-5ffb2a68816e>, line 1)

In [ ]:
-cnn best 1-12

cutter = [
    'DHI', 
    'DNI', 
#     'WS', 
#     'RH', 
#     'T', 
#     'Hour_sin',
#     'Hour_cos', 
    'GHI', 
#     'TARGET_ROLLING_MEAN_3_shift_1',
#     'TARGET_ROLLING_MEAN_5_shift_2', 
#     'TARGET_ROLLING_MEAN_11_shift_5',
#     'TARGET_ROLLING_MEAN_23_shift_11', 
#     'TARGET_ROLLING_MEAN_47_shift_23',
    'scaled_TARGET',
    "TARGET",
]
train_df = train_df[cutter]
valid_df = valid_df[cutter]
test_df = test_df[cutter]

import src.model.multiple_output.convolution
import importlib
importlib.reload(src.model.multiple_output.convolution)


days = 7

one_days_window_label_columns = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=48 * days,
    label_width=96,
    shift=96,
    sequence_stride=1,
    label_columns=["TARGET"]
    
)
submission_df = load_submission_data()
_, predict_df = load_basic_preprocessed_predict("minmax")
predict_df["scaled_TARGET"] = predict_df["TARGET"]
predict_df = predict_df[cutter]
predict_df.drop("TARGET", axis=1, inplace=True)
predict_df = load_test_features(predict_df, 48 * days)

evaluate_dict = {}

for i in range(1, 10):
    q = i/10
    print(q)
    
    conv_various = Convolution2DVarious(days, 96, train_df.shape[-1] - 1)

    compile_and_fit_with_pinball_loss(conv_various, one_days_window_label_columns, q)
    evaluate_dict[q] = conv_various.evaluate(one_days_window_label_columns.test, verbose=0)
    predict_np = predict_df.reshape(-1, days, 48, train_df.shape[-1] - 1)
    pred_y = conv_various.predict(predict_np)[:, :, -1]
    submission_df[f"q_{q}"] = pred_y.reshape(-1)
